### 서울 시민 금융 데이터를 활용한 탄소 배출량 및 폐기물 분석 시각화 고도화

> "소비 데이터를 토대로 탄소 + 폐기물 발생량의 특성 파악"
    
- Carbon Emission from Spending : C.E.P
- 탄소 배출량이 높은 품목을 리스트화 
- 탄소 배출량 및 폐기물 배출량 데이터
- 서울시민의 업종별 카드소비 패턴 데이터와 타 금융의 소비 패턴 데이터


### 사용한 실제 데이터 리스트 링크

1. https://bigdata.seoul.go.kr/data/selectSampleData.do?r_id=P213&sample_data_seq=324&tab_type=&sch_cate=90&file_id=&sch_text=&sch_order=U&currentPage=1

2. https://bigdata.seoul.go.kr/data/selectSampleData.do?r_id=P213&sample_data_seq=318&tab_type=&sch_cate=90&file_id=&sch_text=&sch_order=U&currentPage=1

3. 

In [4]:
# module install

%pip install numpy
%pip install pandas

Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 20.9 MB/s eta 0:00:00m eta 0:00:010:0101
  Using cached pytz-2022.2.1-py2.py3-none-any.whl (500 kB)
Note: you may need to restart the kernel to use updated packages.


In [23]:

# 사전 세팅 및 import
# To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" # "all" 이면, jupyter output 여러개, 한 꺼 번에 볼 수 있음 

import numpy as np
import pandas as pd

DATA_PATH: str = "./datas/cep_data"
FILE_LIST = os.listdir(DATA_PATH) # 해당 directory에 file name을 모두 불러와 List로 저장
FILE_LIST

def get_df_from_target_data(file_idx: int):
    try:
        return pd.read_csv(DATA_PATH + "/" + FILE_LIST[file_idx], index_col = False, encoding='utf-8')
    except UnicodeDecodeError:
        return pd.read_csv(DATA_PATH + "/" + FILE_LIST[file_idx], index_col = False, encoding='cp949')


['롯데멤버스_상품분류표.csv',
 '구매지역및구매자_지역코드.csv',
 '블록별 시간대별소액결제건수 카드소비패턴.csv',
 '카드소비 업종코드.csv',
 '판매지기준 행정동별 상품판매.csv',
 '판매지별 구매자지역기준 블록별 상품판매.csv',
 '판매지기준 블록별 상품판매.csv',
 '집계구별 일별소비지역별 카드소비패턴.csv',
 '집계구별 일별시간대별 카드소비패턴.csv',
 '판매지별 구매자지역기준 행정동별 상품판매.csv',
 '블록별 성별연령대별 카드소비패턴.csv']

In [34]:

카드소비_업종코드 = get_df_from_target_data(3)
카드소비_업종코드['업종코드(UPJONG_CD)'] = 카드소비_업종코드['업종코드(UPJONG_CD)'].str.upper()
# 카드소비_업종코드
블록별_시간대별소액결제건수_카드소비패턴 = get_df_from_target_data(2)
# 블록별_시간대별소액결제건수_카드소비패턴

롯데멤버스_상품분류표 = get_df_from_target_data(0)
통계청분류 = pd.DataFrame()
통계청분류['통계청분류코드'] = 롯데멤버스_상품분류표['통계청분류코드']
통계청분류['통계청분류명'] = 롯데멤버스_상품분류표['통계청분류명']
통계청분류 = 통계청분류.drop_duplicates()


판매지별_구매자지역기준_블록별_상품판매 = get_df_from_target_data(5)
판매지별_구매자지역기준_블록별_상품판매 = pd.merge(판매지별_구매자지역기준_블록별_상품판매, 통계청분류, how='left', left_on='통계청상품코드(STAT_CD)', right_on='통계청분류코드')
판매지별_구매자지역기준_블록별_상품판매 = 판매지별_구매자지역기준_블록별_상품판매.drop(['통계청상품코드(STAT_CD)'], axis='columns')
판매지별_구매자지역기준_블록별_상품판매

df = pd.merge(블록별_시간대별소액결제건수_카드소비패턴, 카드소비_업종코드, how='left', left_on='서울시민업종코드(UPJONG_CD)', right_on='업종코드(UPJONG_CD)')
df = df.drop(['업종코드(UPJONG_CD)'], axis='columns')
df


,기준년월(STD_YM),블록코드(BLCK_CD),성별코드(SEX_CD),연령대코드(AGE_CD),시간대코드(TIME_CD),구매자지역(BUYER_AREA),구매고객수(ACC_CNT),구매건수(PURH_CNT),구매금액(PURH_AMT),통계청분류코드,통계청분류명
0,201911,206648,2,2,6,11680720,7,12,77000,I,오락 및 문화
1,201805,225172,1,3,6,11710520,3,1,8000,E,가정용품 및 가사 서비스
2,201905,224659,1,3,6,11710632,1,10,10000,F,보건
3,201805,413188,2,4,4,11710580,3,95,23000,A,식료품 및 비주류음료
4,201803,364511,1,2,4,11680656,6,1,1000,A,식료품 및 비주류음료
...,...,...,...,...,...,...,...,...,...,...,...
495,201809,274538,2,3,2,11650621,1,1,1000,I,오락 및 문화
496,201804,353409,2,3,4,11470510,4,2,139000,A,식료품 및 비주류음료
497,201806,223026,1,4,5,41190,1,3,40000,A,식료품 및 비주류음료
498,201907,8661,2,4,5,11650651,3,2,4000,A,식료품 및 비주류음료


,서울시민업종코드(UPJONG_CD),기준년월(YM),시간대구간(TIME),고객주소블록코드(BLOCK_CD),카드이용금액계(AMT_CORR),소액결제건수(MICRO_PYM),대분류(CLASS1),중분류(CLASS2),소분류(CLASS3)
0,SS017,202006,6,11063,257,25,유통,기타유통,기타유통
1,SS013,201608,6,216137,86,10,유통,할인점/슈퍼마켓,할인점/슈퍼마켓
2,SS016,201802,3,224819,35,15,유통,편의점,편의점
3,SS055,201906,2,19869,10,0,의료,병원,일반병원
4,SS038,201612,5,28382,10,5,여행/교통,여행,항공
...,...,...,...,...,...,...,...,...,...
495,SS017,201911,5,366735,5,5,유통,기타유통,기타유통
496,SS016,201803,2,155750,10,30,유통,편의점,편의점
497,SS081,202103,3,366426,5,5,스포츠/문화/레저,스포츠/문화/레저,실내골프
498,SS069,201707,3,214935,70,10,전자상거래,전자상거래,결제대행(PG)
